### Reqirements
* #### You need to install module future, manual importing from \_\_future\_\_ is at your convenience
* #### For hdf data import you need pytables too which is not default installed with Anaconda

### Batch execution
* #### ```batch_animal=msaxxyy_z jupyter nbconvert Stat.ipynb --to=html --execute --ExecutePreprocessor.timeout=-1 --output=xxyy_z_report.html```

In [ ]:
#from future.utils import PY3
import future
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import pandas as pd
import numpy as np
import time, os, warnings, imp, itertools
import IPython.display as disp
display = disp.display
import matplotlib as mpl, matplotlib.pyplot as plt
import scipy.stats as stats
zscore, describe = stats.mstats.zscore, stats.describe
import datetime
dt, td = datetime.datetime, datetime.timedelta

%matplotlib inline

In [ ]:
import ca_lib as la
imp.reload(la)

In [ ]:
from os import environ
batch_animal = environ.get('batch_animal', None)

## Load files

In [ ]:
basedir = '../_share/Losonczi/'

# Display database folders
display(os.listdir(basedir))

# Select animal
if batch_animal is None:
    #animal = 'msa0216_4'; FPS = 8
    animal = 'msa0316_1'; FPS = 8
    #animal = 'msa0316_3'; FPS = 8
    #animal = 'msa0316ag_1'; FPS = 8
    #animal = 'msa0915_1'; FPS = 30
    #animal = 'msa0915_2'; FPS = 30
    #animal = 'msa1215_1'; FPS = 30
else:
    FPS = None
    animal = batch_animal

print ('selecting',animal)

# List dir
mydir = os.path.join(basedir,animal)
os.listdir(mydir)

In [ ]:
# Available trials and ROIs
data = la.load_files(mydir)
if (FPS is not None) and (data.FPS != FPS):
    warnings.warn('FPS indication might be wrong.')
print (data.raw.shape, '\n', data.trials, '\n', data.rois)

In [ ]:
# Post-Learning may repeat session_num therefore an additional index,
# day_num is created. See msa0316_1.
# It seems though that Pre-Learning and Learning treats session_num as documented.
display(data.experiment_traits.head())
display(data.experiment_traits[data.experiment_traits['day_leap']])

## Experiment protocol configurations

In [ ]:
def settings_summary(data):
    et_store, et_disp = [], []
    experiment_traits = data.experiment_traits
    et_disp = la.df_epoch(experiment_traits.groupby(la.display_learning).size().to_frame(name=animal))
    et_store = experiment_traits.groupby(la.sort_learning).size().to_frame(name=animal)
    disp.display(disp.HTML('<font color="red">ATTENTION, </font>for later conformity we store columns in a <b>different order</b>: %s !!!'%la.sort_learning))
    et_disp = et_disp.fillna('-')
    et_store = et_store
    display(la.df_epoch(et_disp))
    return et_store

In [ ]:
data.et = settings_summary(data)

## Prepare data

In [ ]:
def availability(data):
    # See how many ROIs are available for which frames
    avail_by_frame = (~data.filtered.isnull()).sum() / len(data.trials)
    plt.plot(avail_by_frame)
    plt.xlabel('Camera frame within experiment')
    plt.ylabel('Available ROIs on average')
    
    # See which ROI is available in which trial and for how many frames
    avail_nframes = ((~data.filtered.isnull()).sum(axis=1)).to_frame('nFrames').unstack(fill_value=0)
    
    return avail_by_frame, avail_nframes

In [ ]:
avail_by_frame, avail_nframes = availability(data)

print(avail_nframes.shape)
display(avail_nframes.head())
display(avail_nframes.tail())

### z-scoring

In [ ]:
FPS = data.FPS
data.z_spike = la.pd_zscore_by_roi(data.spike, FPS, -2*FPS, axis=1)
data.z_filtered = la.pd_zscore_by_roi(data.filtered, FPS, -2*FPS, axis=1).sort_index()
data.z_raw = la.pd_zscore_by_roi(data.raw, FPS, -2*FPS, axis=1).sort_index()
data.z_lick = la.pd_zscore_clip(data.lick, FPS, -2*FPS, axis=1)

### Averaging (integrating)
Spiking is "True" in the [intervals) given in transients_data.hc5

In [ ]:
mymean = pd.DataFrame.mean
mystd = pd.DataFrame.std

### Averaging in 5" bins

In [ ]:
# define 5sec boundaries and bin centers
data.bsections = (np.arange(0,60,5)*FPS).astype(int)
data.bcenters = ((data.bsections[1:]+data.bsections[:-1])/2).astype(int)

# short names
FPS = data.FPS
bsections = data.bsections
bcenters = data.bcenters

# calculate and store
data.zb_spike = la.pd_aggr_col(data.z_spike, mymean, bsections, bcenters)
data.zb_filtered = la.pd_aggr_col(data.z_filtered, mymean, bsections, bcenters)
data.zb_raw = la.pd_aggr_col(data.z_raw, mymean, bsections, bcenters)
data.zb_lick = la.pd_aggr_col(data.z_lick, mymean, bsections, bcenters)

data.b_spike = la.pd_aggr_col(data.spike, mymean, bsections, bcenters)
data.b_data = la.pd_aggr_col(data.filtered, mymean, bsections, bcenters)
data.b_raw = la.pd_aggr_col(data.raw, mymean, bsections, bcenters)
data.b_lick = la.pd_aggr_col(data.lick, mymean, bsections, bcenters)

### Averaging within phases

In [ ]:
# define phase boundaries and bin centers
data.asections = data.event_frames.astype(int)
data.acenters = ((data.asections[1:]+data.asections[:-1])/2).astype(int)

# short names
FPS = data.FPS
asections = data.asections
acenters = data.acenters

# calculate and store
data.za_spike = la.pd_aggr_col(data.z_spike, mymean, asections, acenters)
data.za_filtered = la.pd_aggr_col(data.z_filtered, mymean, asections, acenters)
data.za_raw = la.pd_aggr_col(data.z_raw, mymean, asections, acenters)
data.za_lick = la.pd_aggr_col(data.z_lick, mymean, asections, acenters)

data.a_spike = la.pd_aggr_col(data.spike, mymean, asections, acenters)
data.a_data = la.pd_aggr_col(data.filtered, mymean, asections, acenters)
data.a_raw = la.pd_aggr_col(data.raw, mymean, asections, acenters)
data.a_lick = la.pd_aggr_col(data.lick, mymean, asections, acenters)

### Licking statistics

In [ ]:
# parameters (DoD of lick rate, the boundary of a p<0.005 set for spiking)
max_lick_rate = 20
data.z_spike_threshold = 5.0/np.sqrt(len(data.rois))

# short names
FPS = data.FPS
asections = data.asections
acenters = data.acenters

# The histogram shape justifies putting the threshold at the half maximum
c,b = np.histogram(data.lick.values.ravel(),range=(0,max_lick_rate),bins=max_lick_rate)
data.lick_threshold = (np.argmax(c[1:])+1.5)/2
print(data.lick_threshold)

# Licking rate and fill factor by phase
data.lick_rate_mean = la.pd_aggr_col(data.lick, mymean, asections, acenters)
data.lick_rate_std = la.pd_aggr_col(data.lick, mystd, asections, acenters)
data.lick_time_mean = la.pd_aggr_col((data.lick>data.lick_threshold).astype(float),
                                mymean, asections, acenters)
data.lick_time_std = la.pd_aggr_col((data.lick>data.lick_threshold).astype(float),
                                mystd, asections, acenters)

# Show histogram
plt.figure()
plt.xlim(0,max_lick_rate)
plt.xlabel('lick rate (1/s)')
plt.semilogy()
#plt.hist(data.lick.values.ravel(),log=True,range=(0,max_lick_rate),bins=max_lick_rate)
plt.bar(b[:-1],c,b[1]-b[0],1)
plt.plot(data.lick_threshold,2,'y*',ms=15)

### Triggers

In [ ]:
# short names
experiment_traits = data.experiment_traits
FPS = data.FPS

# Lick triggers
data.lick_triggers_rise, data.lick_triggers_fall = la.trigger_find_pd(data.lick, data.lick_threshold)
print ('Lick threshold crosses:', data.lick_triggers_rise.shape, data.lick_triggers_fall.shape)
print ('Port was present in %d trials.'%data.experiment_traits[data.experiment_traits['port']=='W+'].shape[0])

# Spike triggers
data.spike_triggers_rise, data.spike_triggers_fall = la.trigger_find_pd(data.z_spike.mean(level='time'), data.z_spike_threshold)
print ('Spike threshold crosses:', data.spike_triggers_rise.shape, data.spike_triggers_fall.shape)

# Event triggers
data.csp_triggers_rise, data.csp_triggers_fall, data.csp_triggers_allow = la.trigger_enable_pd(
    experiment_traits[experiment_traits['context']=='CS+'],
    data.event_frames[1], data.event_frames[2])
data.csm_triggers_rise, data.csm_triggers_fall, data.csm_triggers_allow = la.trigger_enable_pd(
    experiment_traits[experiment_traits['context']=='CS-'],
    data.event_frames[1], data.event_frames[2])

data.us_triggers_rise, data.us_triggers_fall, data.us_triggers_allow = la.trigger_enable_pd(
    experiment_traits[experiment_traits['puffed']=='A+'],
    data.event_frames[3], data.event_frames[4])

data.trace_triggers_rise, data.trace_triggers_fall, data.traca_triggers_allow = la.trigger_enable_pd(
    experiment_traits[experiment_traits['context']!='Baseline'],
    data.event_frames[2], data.event_frames[3])

In [ ]:
# Motivation
data['a_lick'].iloc[:,0] > data['lick_threshold']
# Fear
data['a_lick'].iloc[:,2] < data['lick_threshold']
# Sensible
data['a_lick'].iloc[:,3] < data['lick_threshold']

In [ ]:
data['a_lick']

In [ ]:
data['b_lick']

# Plot

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

class helpmultipage(object):
    def __init__(self, filename):
        self.filename = filename
        self.isopen = False
        self.open()
        
    def __del__(self):
        self.close()
        
    def savefig(self, dpi=None):
        if self.isopen:
            self.pp.savefig(dpi=dpi)

    def open(self):
        if (~self.isopen) and len(self.filename):
            self.pp = PdfPages(self.filename)
            self.isopen = True
        
    def close(self):
        if self.isopen:
            self.pp.close()
        self.isopen = False

#### Explanatory figure

In [ ]:
def explain_figures(data):
    import matplotlib.patches as mpatches
    from matplotlib.collections import PatchCollection
    center = (data.event_frames[1:]+data.event_frames[:-1]) /2
    left = data.event_frames
    width = data.event_frames[1:]-data.event_frames[:-1]
    vcenter = 0.0
    vstart = -0.5

    def label90(x,y,text):
        ax.text(x, y, text, ha="center", va="center", family='sans-serif', size=14, rotation=90)

    fig, (empty, ax) = plt.subplots(2,1,figsize=(6,8))
    fig.suptitle('Explanatory figure',fontsize=16)
    fig.tight_layout(pad=3)
    empty.axis('off')
    
    ax.set_xlabel('Camera frame')
    ax.set_ylabel('z-scored activity')
    ax.set_ylim(vstart,vstart+1)
    ax.plot(data.z_spike.mean(axis=0)+0.00, label="(CategoryA, True): #trials", c=(1,1,0))
    ax.plot(data.z_spike.mean(axis=0)+0.02, label="(CategoryB, True): #trials", c=(.5,1,.5))
    ax.plot(-data.z_spike.mean(axis=0)+0.00, label="(CategoryA, False): #trials", c=(1,.8,1))
    ax.plot(-data.z_spike.mean(axis=0)+0.02, label="(CategoryB, False): #trials", c=(.5,1,1))
    patches = []
    # mark delay
    label90(center[0], vcenter, 'excitation by\nshowing water')
    # mark CS
    rect = mpatches.Rectangle((left[1],vstart), width[1], 1, ec="none")
    patches.append(rect)
    label90(center[1], vcenter, 'CS± if tone\n"Baseline" otherwise')
    # mark delay
    label90(center[2], vcenter, 'trace = delay')
    # mark UC
    rect = mpatches.Rectangle((left[3],vstart), width[3], 1, ec="none")
    patches.append(rect)
    label90(center[3], vcenter, 'UC if any')
    # mark water
    ax.text((left[0]+left[3])/2, vstart, "water port present\niff allowed to lick",
            ha="center", va="bottom", family='sans-serif', size=14, bbox=dict(boxstyle="DArrow", pad=0.0, fc='c'))

    # show event boundaries
    for sep in data.event_frames[:-1]:
        ax.axvline(x=sep, ymin=0.0, ymax = 1.0, linewidth=1, color='k')
    colors = np.linspace(0, 1, len(patches))
    collection = PatchCollection(patches, cmap=plt.cm.hsv, alpha=0.1)
    collection.set_array(np.array(colors))
    ax.add_collection(collection)

    # align legend
    leg = ax.legend(loc='lower center', title="Category name, Condition name",
                   bbox_to_anchor=(0.5, 1.1))
    leg.get_title().set_fontsize('large')
    leg.get_title().set_fontweight('bold')
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', UserWarning)
        fig.show()
    return fig

In [ ]:
pp = helpmultipage('%s_explanatory.pdf'%animal)
fig = explain_figures(data)
pp.savefig()
pp.close()

### Learning progress

In [ ]:
def learning_chart(data):
    fig, ax = plt.subplots(len(data.trials),1,figsize=(10,0.6*len(data.trials)), sharex=True, sharey=True)
    fig.tight_layout(h_pad=0.1)
    ind = np.arange(0,5)
    width, height, spacing = 1, 1.2, 10
    label_df = data.experiment_traits.replace('Baseline','B.L.')
    for i, trial in enumerate(data.trials):
        pos = ind+2*spacing
        unit = 2.0*data.lick_threshold
        # need to use .values because integer colummn indices cause confusion
        mea, err = data.lick_rate_mean.loc[trial].values/unit, data.lick_rate_std.loc[trial].values/unit
        rects1 = ax[i].bar(pos, mea, width, color='r', yerr=err)
        pos = ind+3*spacing
        mea = data.lick_time_mean.loc[trial].values
        rects2 = ax[i].bar(pos, mea, width, color='b')
        ax[i].set_xlim(xmin=0)
        ax[i].set_ylim(ymin=0, ymax=height)
        ax[i].set_yticks([0,0.5,1])
        la.draw_conditions(ax[i],label_df,trial,data.FPS,loc='lower left',screen_width=0.5, height=height, cw=[0.25, 0.15, 0.15, 0.15, 0.15, 0.15],fontsize=12)
    ax[-1].set_xticks([spacing, 2.2*spacing, 3.2*spacing])
    ax[-1].set_xticklabels(['Conditions', 'Licking rate', 'Licking time'])
    return fig

In [ ]:
pp = helpmultipage('%s_protocol.pdf'%animal)
fig = learning_chart(data)
pp.savefig()
pp.close()

## Population averages

In [ ]:
pp = helpmultipage(animal+'_pop.pdf')

In [ ]:
la.plot_data(data, [data.spike, data.filtered, data.lick],
             ['Spiking', 'Ca-levels', 'Licking'])
pp.savefig()

### Single criterion
* comments

In [ ]:
grp = [['context'],['learning_epoch'],['port'],['puffed']]
la.plot_data(data, [data.z_spike, data.z_filtered, data.lick], 
             ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
             grp, title='Population activity')
pp.savefig()
la.plot_data(data, [data.zb_spike, data.zb_filtered, data.b_lick],
             ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
             grp, title='Population activity binned', div=bcenters)
pp.savefig()
la.plot_data(data, [data.za_spike, data.za_filtered, data.a_lick],
             ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
             grp, title='Population activity averaged over events', div=acenters)
pp.savefig()

### Two criteria
* comments

### Three criteria
* comments

### All criteria
* There is no increased population activity for CS+ without puffing. (For mouse 0216_4 the 1 trial with port displays increase during the trace period - why?)
* During learning mouse 0216_4 shows incresed activity during the UC phase for CS-

In [ ]:
grp = ['context','port','puffed']
la.plot_epochs(data, [data.z_spike, data.z_filtered, data.lick], 
             ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
             grp, title='Population activity')
pp.savefig()
la.plot_epochs(data, [data.zb_spike, data.zb_filtered, data.b_lick],
             ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
             grp, title='Population activity binned', div=bcenters)
pp.savefig()
la.plot_epochs(data, [data.za_spike, data.za_filtered, data.a_lick],
             ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
             grp, title='Population activity averaged over events', div=acenters)
pp.savefig()

### Activities conditional on epoch

In [ ]:
def plot_by_epoch(epoch):
    experiment_c = data.experiment_traits[data.experiment_traits.loc[:,'learning_epoch']==epoch]
    spike_c = data.z_spike.reindex(experiment_c.index, level='time')
    data_c = data.z_filtered.reindex(experiment_c.index, level='time')
    raw_c = data.z_raw.reindex(experiment_c.index, level='time')
    lick_c = data.lick.reindex(experiment_c.index)
    print (experiment_c.shape, spike_c.shape)
    spike_ca = la.pd_aggr_col(spike_c, mymean, asections, acenters)
    data_ca = la.pd_aggr_col(data_c, mymean, asections, acenters)
    raw_ca = la.pd_aggr_col(raw_c, mymean, asections, acenters)
    lick_ca = la.pd_aggr_col(lick_c, mymean, asections, acenters)
    print (spike_c.shape, spike_ca.shape)

    grp = [['context','port'],['context','puffed'],['port','puffed']]
    la.plot_data(data, [spike_c, data_c, lick_c],
                 ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
                 grp, title=epoch)
    pp.savefig()
    la.plot_data(data, [spike_ca, data_ca, lick_ca],
                 ['z-scored Spiking', 'z-scored Ca-levels', 'Licking'],
                 grp, title=epoch+' averaged over events', div=acenters)
    pp.savefig()

#### Pre-learning

In [ ]:
plot_by_epoch('Pre-Learning')

#### Learning

In [ ]:
plot_by_epoch('Learning')

#### Post-Learning

In [ ]:
plot_by_epoch('Post-Learning')

In [ ]:
pp.close()

## Activity vector by phases

In [ ]:
def plot_activity_vectors(data, df, title, **kwarg):
    # provide trial IDs by condition (non-unique index)
    etmp = data.experiment_traits.reset_index(drop=True).set_index(la.sort_learning)

    nplot = len(data.et.index)
    ncol = 14
    nrow = int(np.ceil(len(data.et.index)/float(ncol)))
    
    fig, ax = plt.subplots(nrow,ncol,figsize=(2*ncol,1+10*nrow),squeeze=False,sharey=True)
    fig.tight_layout(pad=3, h_pad=3, rect=[0,0,1,0.8])
    fig.suptitle(title,fontsize=16)
    for i, cond in enumerate(data.et.index):
        icol = i%ncol
        irow = int((i-icol)/ncol)
        sel = etmp.loc[cond,'timestr']
        tmp = df.loc[sel.tolist(),aggr].unstack('time')
        img = ax[irow,icol].matshow(tmp.values,origin='lower', **kwarg)
        ax[irow,icol].xaxis.set_ticks_position('bottom')
        ax[irow,icol].set_title('\n'.join(cond))
        ax[irow,icol].set_ylabel('Unit ID')
        ax[irow,icol].set_xlabel('Trial')
    #cax,kw = mpl.colorbar.make_axes([axis for axis in ax.flat])
    cax = ax[-1,-1]
    plt.colorbar(img,ax=cax)#ax=cax,**kw)

In [ ]:
pp = helpmultipage(animal+'_activation_sp.pdf')

for p,aggr in enumerate(data.a_spike.columns):
    title='Spikes, Phase: %s'%la.phases[p]
    plot_activity_vectors(data, data.a_spike, title,vmin=0,vmax=1)
    pp.savefig()

pp.close()

In [ ]:
pp = helpmultipage(animal+'_activation_ca.pdf')

for p,aggr in enumerate(data.za_filtered.columns):
    title = 'Ca-Signal, Phase: %s'%la.phases[p]
    plot_activity_vectors(data, data.za_filtered, title, vmin=-3, vmax=3)
    pp.savefig()    
pp.close()

### An example of spiking
The first 1 second of the recording seems missing

In [ ]:
# Triggers
trig_list_data = [data.lick_triggers_rise, data.lick_triggers_fall,
                  data.spike_triggers_rise, data.spike_triggers_fall]
trig_list_sign = ['o', 's', '^', 'v']
trig_list_color = ['b', 'y', 'r', 'g']

In [ ]:
def draw_firing(ax, data, idx, settings, seismic=False, show_nan=False, pos=-20):
    #experiment_id = settings['timestr']
    fig.suptitle('%s: session %s, day %s\n'%(idx,settings['session_num'],settings['day_num'])+
                 ', '.join(la.sort_learning)+': #context in epoch, #day',fontsize=16)
    if seismic:
        la.draw_levels(ax, data.z_filtered, idx, data.FPS, data.roi_df)
    else:
        la.draw_transients(ax, data.transients, idx, data.FPS, data.roi_df)
    if show_nan:
        la.draw_spiking_nan(ax, data.spike, idx, data.rois.values)

def draw_signals(ax, data, idx, settings, seismic=False, show_nan=False, pos=-20):
    experiment_id = settings['timestr']
    la.draw_population(ax, data.z_filtered, idx, pos=pos, c='y', label='population Ca-signal')
    la.draw_population(ax, data.z_spike, idx, pos=pos, threshold=data.z_spike_threshold, label='population z-spike count')
    la.draw_licking(ax, data.lick, idx, pos=pos-20, threshold=data.lick_threshold, label='licking')
    la.draw_triggers(ax, trig_list_data, idx, -5, trig_list_sign, c=trig_list_color)
    la.draw_conditions(ax, data.experiment_traits, experiment_id, data.FPS, height=20)
    return ax

In [ ]:
# Show an example
idx, settings = data.experiment_traits.index[9], data.experiment_traits.iloc[9,:]
fig, ax = plt.subplots(1,1,figsize=(16,10))
ax.set_ylim(ymin=-60,ymax=len(data.rois)+1)
draw_firing(ax, data, idx, settings, True)
draw_signals(ax, data, idx, settings, True)
ax.legend()

In [ ]:
# Show an example
idx, settings = data.experiment_traits.index[9], data.experiment_traits.iloc[9,:]
fig, ax = plt.subplots(1,1,figsize=(16,10))
ax.set_ylim(ymin=-60,ymax=len(data.rois)+1)
draw_firing(ax, data, idx, settings, False, True)
draw_signals(ax, data, idx, settings, False, True)
ax.legend()

In [ ]:
pp = helpmultipage(animal+'_firing.pdf')

xmax = data.transients.loc[:,['stop_frame']].max().values

for idx, settings in data.experiment_traits.iterrows():
    fig, ax = plt.subplots(1,1,figsize=(16,10))
    ax.set_xlim(xmax=xmax)
    ax.set_ylim(ymin=-60,ymax=len(data.rois)+1)
    draw_firing(ax, data, idx, settings)
    draw_signals(ax, data, idx, settings)
    pp.savefig()
    plt.close(fig)
    
pp.close()

### Pattern matching

In [ ]:
prog_update = 1468579493
print ("%.0f"%time.time())

In [ ]:
pattdb_file = 'pattdb_'+animal+'.h5'
if 'pattdb' in locals():
    pattdb.close()
    del pattdb
if (not la.test_hdf(pattdb_file)) or (os.path.getmtime(pattdb_file)<prog_update):
    with pd.HDFStore(pattdb_file, mode='w') as pattdb:
        for method,sel in itertools.product(['match','correlate'],['sp','ca']):
            print(method,sel)
            df = data.spike if sel == 'sp' else data.z_filtered.reindex(data.mirow)
            key = '/'.join((method,sel,'lick_rise_csp'))
            pattdb[key] = la.search_pattern(df, data.lick_triggers_rise, data.trials,
                                            data.FPS, trigger_allow=data.csp_triggers_allow)
            key = '/'.join((method,sel,'lick_fall_csp'))
            pattdb[key] = la.search_pattern(df, data.lick_triggers_fall, data.trials,
                                            data.FPS, trigger_allow=data.csp_triggers_allow)
            key = '/'.join((method,sel,'csp_rise'))
            pattdb[key] = la.search_pattern(df, data.csp_triggers_rise, data.trials,
                                            data.FPS, trigger_allow=data.csp_triggers_allow)
            key = '/'.join((method,sel,'us_rise'))
            pattdb[key] = la.search_pattern(df, data.us_triggers_rise, data.trials, data.FPS)
pattdb = pd.HDFStore(pattdb_file, mode='r')

In [ ]:
data.z_patt = {}
for key in pattdb.keys():
    if key[0] == '/':
        key = key[1:]
    data.z_patt[key] = la.nan_zscore(pattdb[key])
pattdb

In [ ]:
def show_detections(ax, z_patt, method, sel, idx, ids, names, colors):
    threshold = [-3, 3]
    zoom = 3
    for i, id1 in enumerate(ids):
        la.draw_licking(ax, z_patt['/'.join((method,sel,id1))], idx, pos=-20, c=colors[i],
                threshold=threshold, zoom=zoom, label='%s: %s'%(sel, names[i]))
        threshold = None
    return ax

In [ ]:
# Show an example
method = 'match' # 'match', 'correlate'
sel = 'ca' # 'ca', 'sp'
idx, settings = data.experiment_traits.index[13], data.experiment_traits.iloc[13,:]
fig, ax = plt.subplots(1,1,figsize=(16,10))
ax.set_ylim(ymin=-80,ymax=len(data.rois)+1)
draw_firing(ax, data, idx, settings, False, True, pos=-40)
show_detections(ax, data.z_patt, method, sel, idx, ['lick_rise_csp','lick_fall_csp','csp_rise','us_rise'],
                ['CS+ lick start', 'CS+ lick end', 'CS+ start', 'US start'], ['g', 'c', 'orange', 'r'])
draw_signals(ax, data, idx, settings, False, True, pos=-40)
ax.legend()

In [ ]:
for method,sel in itertools.product(['match','correlate'], ['sp','ca']):
    print (method,sel)

    pp = helpmultipage(animal+'_triggers_%s_%s.pdf'%(method,sel))
    zoom = 4

    xmax = data.transients.loc[:,['stop_frame']].max().values

    for idx, settings in data.experiment_traits.iterrows(): #et3.iterrows():
        fig, ax = plt.subplots(1,1,figsize=(16,10))
        ax.set_xlim(xmax=xmax)
        ax.set_ylim(ymin=-80,ymax=len(data.rois)+1)
        draw_firing(ax, data, idx, settings, False, True, pos=-40)
        show_detections(ax, data.z_patt, method, sel, idx, ['lick_rise_csp','lick_fall_csp','csp_rise','us_rise'],
                ['CS+ lick start', 'CS+ lick end', 'CS+ start', 'US start'], ['g', 'c', 'orange', 'r'])
        draw_signals(ax, data, idx, settings, False, True, pos=-40)
        ax.legend()
        pp.savefig()
        plt.close(fig)

    pp.close()

## Peri-event plots

In [ ]:
def list_peri_3a(data, df, title=None):
    '''Plot collection: CS+ US'''
    ret = [] # df, trig, allow, disable, title
    ret.append([df, data.rois, data.lick_triggers_rise, None, None, 'Lick rise'])
    ret.append([df, data.rois, data.lick_triggers_fall, None, None, 'Lick fall'])
    ret.append([df, data.rois, data.lick_triggers_rise, data.csp_triggers_allow, None, 'Lick rise CS+'])
    ret.append([df, data.rois, data.lick_triggers_fall, data.csp_triggers_allow, None, 'Lick fall CS+'])
    ret.append([df, data.rois, data.csp_triggers_rise, None, None, 'CS+ start'])
    ret.append([df, data.rois, data.csp_triggers_fall, None, None, 'CS+ end'])
    ret.append([df, data.rois, data.lick_triggers_rise, data.us_triggers_allow, None, 'Lick rise US'])
    ret.append([df, data.rois, data.lick_triggers_fall, data.us_triggers_allow, None, 'Lick fall US'])
    ret.append([df, data.rois, data.us_triggers_rise, None, None, 'US start'])
    ret.append([df, data.rois, data.us_triggers_fall, None, None, 'US end'])
    return ret

In [ ]:
def list_peri_3b(data, df, title=None):
    '''Plot collection: CS+ US'''
    ret = [] # df, trig, allow, disable, title
    ret.append([df, data.rois, data.lick_triggers_rise, None, None, 'Lick rise'])
    ret.append([df, data.rois, data.lick_triggers_fall, None, None, 'Lick fall'])
    ret.append([df, data.rois, data.lick_triggers_rise, data.csp_triggers_allow, None, 'Lick rise CS+'])
    ret.append([df, data.rois, data.lick_triggers_fall, data.csp_triggers_allow, None, 'Lick fall CS+'])
    ret.append([df, data.rois, data.csp_triggers_rise, None, None, 'CS+ start'])
    ret.append([df, data.rois, data.csp_triggers_fall, None, None, 'CS+ end'])
    ret.append([df, data.rois, data.lick_triggers_rise, data.csm_triggers_allow, None, 'Lick rise CS-'])
    ret.append([df, data.rois, data.lick_triggers_fall, data.csm_triggers_allow, None, 'Lick fall CS-'])
    ret.append([df, data.rois, data.csm_triggers_rise, None, None, 'CS- start'])
    ret.append([df, data.rois,data. csm_triggers_fall, None, None, 'CS- end'])
    return ret

In [ ]:
# Show an example
fig=la.plot_peri_collection(list_peri_3a(data, data.spike),'Spiking',
                            (-2*data.FPS,6*data.FPS),combine=False)
fig=la.plot_peri_collection(list_peri_3a(data, data.spike),'Spiking',
                            (-2*data.FPS,6*data.FPS))

In [ ]:
pp = helpmultipage(animal+'_peri1.pdf')
for epoch in la.epochs.values:
    experiment_c = data.experiment_traits[data.experiment_traits.loc[:,'learning_epoch']==epoch]
    spike_c = data.spike.reindex(experiment_c.index, level='time')
    #data_c = data.z_filtered.reindex(experiment_c.index, level='time')
    fig=la.plot_peri_collection(list_peri_3a(data,spike_c),'%s Spiking on US'%epoch,
                                (-2*data.FPS,6*data.FPS),combine=False)
    pp.savefig()
    plt.close(fig)
    fig=la.plot_peri_collection(list_peri_3b(data,spike_c),'%s Spiking on CS+/-'%epoch,
                                (-2*data.FPS,6*data.FPS),combine=False)
    pp.savefig()
    plt.close(fig)
for epoch in la.epochs.values:
    experiment_c = data.experiment_traits[data.experiment_traits.loc[:,'learning_epoch']==epoch]
    #spike_c = data.spike.reindex(experiment_c.index, level='time')
    data_c = data.z_filtered.reindex(experiment_c.index, level='time')
    fig=la.plot_peri_collection(list_peri_3a(data,data_c),'%s z-scored Ca-level on US'%epoch,
                                (-2*data.FPS,6*data.FPS),combine=False)
    pp.savefig()
    plt.close(fig)
    fig=la.plot_peri_collection(list_peri_3b(data,data_c),'%s z-scored Ca-level on CS+/-'%epoch,
                                (-2*data.FPS,6*data.FPS),combine=False)
    pp.savefig()
    plt.close(fig)
pp.close()

In [ ]:
pp = helpmultipage(animal+'_peri2.pdf')
for epoch in la.epochs.values:
    experiment_c = data.experiment_traits[data.experiment_traits.loc[:,'learning_epoch']==epoch]
    spike_c = data.spike.reindex(experiment_c.index, level='time')
    #data_c = data.z_filtered.reindex(experiment_c.index, level='time')
    fig=la.plot_peri_collection(list_peri_3a(data,spike_c),'%s Spiking on US'%epoch,
                               (-2*data.FPS,6*data.FPS))
    pp.savefig()
    plt.close(fig)
    fig=la.plot_peri_collection(list_peri_3b(data,spike_c),'%s Spiking on CS+/-'%epoch,
                                (-2*data.FPS,6*data.FPS))
    pp.savefig()
    plt.close(fig)
for epoch in la.epochs.values:
    experiment_c = data.experiment_traits[data.experiment_traits.loc[:,'learning_epoch']==epoch]
    #spike_c = data.spike.reindex(experiment_c.index, level='time')
    data_c = data.z_filtered.reindex(experiment_c.index, level='time')
    fig=la.plot_peri_collection(list_peri_3a(data,data_c),'%s z-scored Ca-level on US'%epoch,
                               (-2*data.FPS,6*data.FPS))
    pp.savefig()
    plt.close(fig)
    fig=la.plot_peri_collection(list_peri_3b(data,data_c),'%s z-scored Ca-level on CS+/-'%epoch,
                               (-2*data.FPS,6*data.FPS))
    pp.savefig()
    plt.close(fig)
pp.close()

## Individual ROIs
* since there are many of them, save figure to pdf
* THIS WILL <font color="red">TAKE A WHILE</font>, consider testing with a small range

In [ ]:
def plot_roi(filename, data, dfs, names, grp, title_template, by_epoch=False, div=None, fill=None):
    pp = PdfPages(filename)
    for i in range(0,len(data.rois)):
        dfc = []
        for df in dfs:
            dfc.append(df.loc[(slice(None),data.rois[i]),:])
        if by_epoch:
            fig = la.plot_epochs(data, dfc, names, grp, title=title_template%(i,data.rois[i]), div=div, fill=fill)
        else:
            fig = la.plot_data(data, dfc, names, grp, title=title_template%(i,data.rois[i]), div=div, fill=fill)
        pp.savefig()
        plt.close(fig)
    pp.close()

In [ ]:
if batch_animal is None:
    raise ValueError("You don't want to run this automatically")

#### Raw data

In [ ]:
plot_roi(animal+'_roiAcrit.pdf', data, [data.spike, data.filtered],
         ['Spiking', 'Ca-level'], ['context','port','puffed'],'ROI %d:\n%s',True)

### Averaging over intervals

#### Intervals aligned to events

In [ ]:
plot_roi(animal+'_avgAcrit.pdf', data, [data.a_spike, data.a_data],
         ['Spiking', 'Ca-level'], ['context','port','puffed'],'ROI %d:\n%s',True,div=acenters, fill='err')

#### Averaging over bins

In [ ]:
plot_roi(animal+'_binAcrit.pdf', data, [data.b_spike, data.b_data],
         ['Spiking', 'Ca-level'], ['context','port','puffed'],'ROI %d:\n%s',True,div=acenters, fill='err')

## Correlations

In [ ]:
# Whether to average trials (assuming identical timing) or rather concatenate them
concatenate = True

In [ ]:
def concat_for_correlation(df, data):
    # Combine information
    ordered = df.reindex(data.mirow, data.icol)
    et1 = data.experiment_traits[data.experiment_traits.loc[:,'session_num']>=0]
    et1 = et1.loc[:,la.sort_learning+['day_num','session_num']]
    ordered = ordered.join(et1, how='inner').reset_index().drop('time', axis=1) # keep roi_id
    ordered = ordered.set_index(la.sort_learning+['roi_id', 'session_num']).sort_index()
    ordered.columns.name='Spike'
    #display(ordered.head())

    # Search for days that contain experiments with same traits and session_num
    # These entries would jeopardize unstacking
    et2 = et1.reset_index(drop=True).set_index(la.sort_learning+['session_num']).sort_index()
    second_occur = et2.index.duplicated()
    set_second = et2.loc[second_occur,'day_num'].unique()
    all_occur = et2.index.get_duplicates()
    set_all = et2.loc[all_occur,'day_num'].unique()
    set_first = np.array(list(set(set_all)-set(set_second)))
    print('Days repeating settings: %s, all conflicted: %s, to be kept: %s'%
          (set_second,set_all,set_first))

    # Filter out second occurrences stored in set2
    if len(set_first):
        ordered = ordered[ordered.loc[:,'day_num'].apply(lambda x: x not in set_first)]
    print('Filtered data:',ordered.shape)

    # Reshape for correlation analysis
    # integer values get converted to float if needed to hold NaN-s
    calendar = ordered['day_num'].unstack(fill_value=0)
    ordered = ordered.drop(['day_num'], axis=1).unstack()
    print('Concatenated data:',ordered.shape)
    return ordered, calendar

In [ ]:
def average_for_correlation(df, data):
    # Combine information
    ordered = df.reindex(data.mirow, data.icol)
    et1 = data.experiment_traits[data.experiment_traits.loc[:,'session_num']>=0]
    et1 = et1.loc[:,la.sort_learning+['day_num','session_num']]
    ordered = ordered.join(et1, how='inner').reset_index().drop('time', axis=1) # keep roi_id
    ordered = ordered.set_index(la.sort_learning+['roi_id', 'session_num']).sort_index()
    ordered.columns.name='Spike'
    #display(ordered.head())

    # Reshape for correlation analysis
    # integer values get converted to float if needed to hold NaN-s
    calendar = ordered['day_num'].unstack(fill_value=0)
    avg_labels = list(ordered.index.names)
    avg_labels.remove('session_num')
    print (avg_labels)
    ordered = ordered.drop(['day_num'], axis=1).mean(level=avg_labels)
    compatibility_index_level =  pd.Index([1], name='session_num')
    ordered.columns = pd.MultiIndex.from_product((ordered.columns, compatibility_index_level),
                                              names=(ordered.columns.name,compatibility_index_level.name))
    print('Averaged data:',ordered.shape)
    return ordered, calendar

In [ ]:
data.ord_correl, data.calendar = concat_for_correlation(data.z_filtered, data)
data.ord_tr_avg, data.calendar = average_for_correlation(data.z_filtered, data)

In [ ]:
if concatenate:
    ord1 = data.ord_correl
    data.corr_method = 'concatenated'
else:
    ord1 = data.ord_tr_avg
    data.corr_method = 'trial-averaged'

In [ ]:
# Set a reference for ordering elements in the matrix
data.key_ref = ('Post-Learning','CS+','W+','A+')
if data.key_ref not in data.et.index:
    warnings.warn('Using alternative reference')
    data.key_ref = ('Learning','CS+','W+','A+')

FPS = data.FPS

time_ref = np.array([15, 40])
data.col_ref = slice(int(time_ref[0]*FPS),int(time_ref[1]*FPS))
sel = ord1.loc[data.key_ref+(slice(None),),data.col_ref]
print(data.key_ref,time_ref,data.col_ref,sel.shape)

# Correlate
corr_df = sel.T.corr()
corr_np = corr_df.fillna(0).values

# Discard invalid series
data.keep = (np.diag(corr_np) == 1.0)
data.corr_np = corr_np[data.keep,:][:,data.keep]

# Show
fig, ax = plt.subplots(1,2, figsize=(12,4))
img = ax[0].matshow(corr_df.values)
img = ax[1].matshow(corr_np)
fig.colorbar(img, ax=ax[1])

In [ ]:
pp = helpmultipage(animal+'_correl_'+data.corr_method+'.pdf')

In [ ]:
# Define an ordering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform, pdist

sq_dist = squareform(1.0-data.corr_np)
corr_link = linkage(sq_dist, 'average')
sel = ord1.loc[data.key_ref+(slice(None),),data.col_ref]

fig, ax = plt.subplots(1,2, figsize=(18,8))
fig.suptitle('Reference for '+animal+' is presented here: '+(', '.join(np.array(data.key_ref)))+
            ' and time '+('..'.join(time_ref.astype(str)))+'s',fontsize=16)
labels = sel.index.get_level_values(4).to_series().reset_index(drop=True)[data.keep]
dendo = dendrogram(corr_link, ax=ax[1], labels=labels.values, leaf_font_size=2.5, orientation='left')
ax[1].set_title('Distance of firing patterns')
data.corr_order = dendo['leaves']
# Show reordered
img = ax[0].matshow(data.corr_np[data.corr_order,:][:,data.corr_order], origin='lower', vmin=-0.8, vmax=1)
ax[0].xaxis.set_ticks_position('bottom')
ax[0].set_title('Ordered correlation matrix', y=1.0)
fig.colorbar(img)
pp.savefig(dpi=600)

In [ ]:
# Calculate under different conditions
phase_start = data.event_frames+data.FPS
phase_end = data.event_frames[1:]-data.FPS

data.mx = {}
data.mi = pd.DataFrame([], index = la.phases, columns = data.et.index).unstack().index
data.ds = pd.DataFrame(columns = data.mi)

keep = data.keep
corr_order = data.corr_order

for (irow,key),(icol,phase) in itertools.product(enumerate(data.et.index),enumerate(la.phases)):
    # Find the pre-learning structure
    col_sel = slice(int(phase_start[icol]),int(phase_end[icol]))
    try:
        sel = ord1.loc[key+(slice(None),),col_sel]
        #print(key,phase,ord1.shape,sel.shape)

        # Correlate
        corr_tmp = sel.T.corr()
        corr_tmp = corr_tmp.fillna(0).values
    except KeyError:
        corr_tmp = []

    # Discard invalid series
    if len(corr_tmp):
        corr_tmp = corr_tmp[keep,:][:,keep][corr_order,:][:,corr_order]
    else:
        corr_tmp = np.nan * np.ones((len(corr_order),len(corr_order)))

    data.mx[key+(phase,)] = corr_tmp
    data.ds[key+(phase,)] = np.ravel(corr_tmp+np.diag(np.nan*np.diag(corr_tmp)))

In [ ]:
# Show under different conditions
num_phases = 3
num_rows = len(data.et.index)
num_cols = num_phases #len(la.phases)

fig, ax = plt.subplots(num_rows,num_cols, figsize=(5*num_cols,5*num_rows))
fig.suptitle('Correlation structure under different conditions: learning_epoch, context, port, puffed\n'+
             '(small number of trials might lead to larger percieved correlation)\n'+
             '(in phases Ready, CS, Trace the conditions A+ and A- should be very similar)',fontsize=16)
#ax = np.ravel(ax)

keep = data.keep
corr_order = data.corr_order

for (irow,key),(icol,phase) in itertools.product(enumerate(data.et.index),enumerate(la.phases[:num_cols])):
    # Find the pre-learning structure
    count = np.nan_to_num(data.et.loc[key,animal])
    try:
        corr_tmp = data.mx[key+(phase,)]
    except KeyError:
        corr_tmp = []

    img = ax[irow,icol].matshow(corr_tmp, origin='lower', vmin=-0.8, vmax=1)
    ax[irow,icol].xaxis.set_ticks_position('bottom')
    ax[irow,icol].set_title('%s, %s: %d'%(key,phase,count))

pp.savefig(dpi=600)
#plt.close(fig)

## Statistics of the correlation coefficients

In [ ]:
num_rows = len(data.et.index)
num_cols = num_phases

fig, ax = plt.subplots(num_rows,num_cols, figsize=(5*num_cols,5*num_rows))
fig.suptitle('Distribution of the above correlation coefficients\n'+
             'for '+animal+' (diagonals excluded)',fontsize=16)
#ax = np.ravel(ax)

for (irow,key),(icol,phase) in itertools.product(enumerate(data.et.index),enumerate(la.phases[:num_cols])):
    count = np.nan_to_num(data.et.loc[key,animal])
    try:
        corr_tmp = data.mx[key+(phase,)]
        corr_tmp = corr_tmp+np.diag(np.nan*np.diag(corr_tmp))
    except KeyError:
        corr_tmp = []

    if len(corr_tmp) and np.sum(corr_tmp>-1.0):
        ax[irow,icol].hist(np.ravel(corr_tmp),range=(-1,1),bins=20)
        ax[irow,icol].set_yscale('log')
    ax[irow,icol].set_title('%s, %s: %d'%(key,phase,count))

pp.savefig()
#plt.close(fig)

In [ ]:
oraculum = True

### Compare correlation coefficient distributions

In [ ]:
def describe_correlation(data, title, has_oraculum):
    '''data columns is MultIndex containing trial settings, has_oraculum iff airpuff is a parameter'''
    fig, ax = plt.subplots(1,1,figsize=(12,16))
    fig.suptitle(title,fontsize=16)
    ax.axis('off')
    if oraculum:
        stat = np.round(data.describe(),4).T
    else:
        # pivot out airpuff to be able to average over it
        stat = np.round(data.stack(level='puffed').describe(),4).T
    ordered = la.df_epoch(stat)
    stat = stat.sort_index()
    cw = np.ones((len(ordered.columns),))
    tab = mpl.table.table(ax, cellText=ordered.values,
             rowLabels=[', '.join(x) for x in ordered.index.values],
             colLabels=ordered.columns.values.astype(str),
             loc='upper right', fontsize=20, colWidths=0.6*cw/np.sum(cw),
             bbox=[0.3,0,0.7,1], cellLoc='center')
    return stat, fig

In [ ]:
def compare_correlation(stat, title, has_oraculum):
    lmi = pd.DataFrame([], index=la.phases[0:num_phases],
            columns = la.legal_conditions if oraculum else la.short_conditions).unstack().index

    fig, ax = plt.subplots(1,1,figsize=(12,16))
    fig.suptitle(title,fontsize=16)
    ax.axis('off')
    cellcolor = np.vectorize(lambda x: 'lightcoral' if x>0.5 else (
                            'lightblue' if x<-0.4 else 'white'))
    c = np.sqrt(stat.mean().loc['count'])
    diff = []
    for epoch1, epoch2 in [('Learning','Pre-Learning'),
                           ('Post-Learning','Pre-Learning'),('Post-Learning','Learning')]:
        try:
            d = (stat.loc[epoch1,'mean']-stat.loc[epoch2,'mean'])/(
                 stat.loc[epoch1,'std']+stat.loc[epoch2,'std'])*2
        except KeyError:
            d = pd.Series([]).reindex(index=lmi)
        d = d.to_frame(name='  -  '.join((epoch1,epoch2)).replace('-Learning','-L'))
        diff.append(d)
    diff = np.round(pd.concat(diff,axis=1),4).reindex(lmi)
    cw = np.ones((3,))
    tab = mpl.table.table(ax, cellText=diff.values,
             cellColours=cellcolor(diff.values),
             rowLabels=[', '.join(x) for x in diff.index.values],
             rowColours=np.repeat(la.legal_colors if oraculum else la.short_colors,num_phases),
             colLabels=diff.columns.values.astype(str),
             loc='upper right', fontsize=32, colWidths=0.6*cw/np.sum(cw),
             bbox=[0.3,0,0.7,1], cellLoc='center')
    tab.set_fontsize(32)
    return diff, fig

In [ ]:
def plot_correlation_bars(stat, title, et, has_oraculum):
    num_rows = num_phases
    num_cols = len(la.epochs)
    # We don't use sharex on purpose: we want to set different tick labels in the subplot columns
    fig, ax = plt.subplots(num_rows, num_cols, figsize=(5*num_cols,5*num_rows), sharey=True)
    fig.suptitle(title,fontsize=16)
    cat = len(la.legal_conditions if has_oraculum else la.short_conditions)
    bars = stat.reset_index().set_index(['phase']+la.sort_learning[0:(4 if has_oraculum else 3)])
    for (irow,phase), (icol,epoch) in itertools.product(enumerate(la.phases[:num_phases]),enumerate(la.epochs)):
        try:
            bar = bars.loc[(phase, epoch),:].reindex(la.legal_conditions if has_oraculum else la.short_conditions)
            if has_oraculum:
                lab = et.loc[epoch].reindex(la.legal_conditions, fill_value=0)
            else:
                lab = et.loc[epoch].sum(level=('context','port')).reindex(la.short_conditions, fill_value=0)
            ax[irow,icol].set_title(epoch)
            ax[irow,icol].set_ylabel(phase)
            low, high = [0]+bar['25%'].fillna(0).tolist(),[0]+bar['75%'].fillna(0).tolist()
            ax[irow,icol].fill_between(np.arange(0,cat+1), low, high, alpha=0.1, interpolate=False, color='grey', edgecolor=None, step='pre')
            ax[irow,icol].bar(range(0,cat),bar['mean'],1,yerr=bar['std'],color=la.legal_colors if has_oraculum else la.short_colors)
            ax[irow,icol].set_xticks(np.arange(0,cat)+0.5)
            if irow+1==num_rows:
                labels = [('%s: %d'%(', '.join(idx),np.nan_to_num(count))) for idx,count in lab.iteritems()]
            else:
                labels = ['%d'%np.nan_to_num(count) for idx,count in lab.iteritems()]
            ax[irow,icol].set_xticklabels(labels, rotation='vertical')

        except KeyError:
            pass
    return fig

### Real value

In [ ]:
data.stat, fig = describe_correlation(data.ds, 'Statistics on the correlation coefficients', oraculum)
pp.savefig()
plt.close(fig)
# count corresponds to the number of elemenets in the correlation matrix
display(la.df_epoch(data.stat))

In [ ]:
data.diff, fig = compare_correlation(data.stat, 'Difference of the above correlation coefficients\n'+
                 'in stdev. units', oraculum)
pp.savefig()
#plt.close(fig)
#display(data.diff)

In [ ]:
plot_correlation_bars(data.stat, 'Distribution of the above coefficients\n'
                      'for '+animal+' (diagonals excluded)', data.et.loc[:,animal], oraculum)
pp.savefig()
plt.close(fig)

### Absolute value

In [ ]:
data.astat, fig = describe_correlation(np.abs(data.ds), 'Statistics on the absolute value\nof the correlation coefficients', oraculum)
pp.savefig()
plt.close(fig)
# count corresponds to the number of elemenets in the correlation matrix
display(la.df_epoch(data.astat))

In [ ]:
data.adiff, fig = compare_correlation(data.astat, 'Difference of the absolute value of the correlation coefficients\n'+
                 'in stdev. units', oraculum)
pp.savefig()
#plt.close(fig)
#display(data.adiff)

In [ ]:
plot_correlation_bars(data.astat, 'Distribution of the absolute value of the correlation coefficients\n'
                      'for '+animal+' (diagonals excluded)', data.et.loc[:,animal], oraculum)
pp.savefig()

## Similarity of correlation matrices

In [ ]:
num_rows = len(data.et.index)
num_cols = num_phases

change = np.zeros((num_cols,num_rows,num_rows))
for icol, phase in enumerate(la.phases[:num_phases]):
    for irow1, key1 in enumerate(data.et.index):
        count1 = data.et.loc[key1,animal]
        for irow2, key2 in enumerate(data.et.index):
            count2 = data.et.loc[key2,animal]
            change[icol,irow1,irow2] = np.linalg.norm(np.ravel(
                    data.mx[key1+(phase,)]-data.mx[key2+(phase,)])/np.size(data.mx[key2+(phase,)]),2)

for icol, phase in enumerate(la.phases[:num_phases]):
    fig, ax = plt.subplots(1,1, figsize=(8,6))
    fig.tight_layout(rect=[0.4,0,0.95,0.55])
    #fig = plt.figure()
    #ax = fig.gca()
    img = ax.matshow(change[icol]+np.diag(np.nan*np.diag(change[icol])),
                     cmap=plt.get_cmap('rainbow'))

    fig.suptitle('Difference between test cases (RMS distance)\nPhase: '
                 +phase,fontsize=16)
    ax.set_xticks(np.array(range(0,num_rows)))
    ax.set_xticklabels(data.et.index.tolist(),rotation=90)
    ax.set_yticks(np.array(range(0,num_rows)))
    ax.set_yticklabels(data.et.index.tolist())

    # without set_yticks
    # ax.set_yticklabels([tuple()]+et.index.values.tolist())
    fig.colorbar(img)
    pp.savefig()

In [ ]:
pp.close()

In [ ]:
la.store_to_hdf('anidb_'+animal+'.h5', data)

data.roi_df['roi_id'].dtype is string
data.colref is slice(n,m,None)
data.transients['sigma'].dtype is object

In [ ]:
test = la.read_from_hdf('anidb_'+animal+'.h5', la.Bunch())